In [4]:
def get_rectangle():
    import random
    width = random.random()
    height = random.random()
    fat = int(width >= height)
    return width, height, fat

import torch
from torch.utils.data import Dataset

class Dataset(Dataset):
    def __inti__(self):
        pass
    def __len__(self):
        return 500
    def __getitem__(self, i):
        width, heigth, fat = get_rectangle()
        x = torch.FloatTensor([width, heigth])
        y = fat
        return x,y
    
dataset = Dataset()
len(dataset), dataset[0]

(500, (tensor([0.1275, 0.3408]), 0))

In [13]:
import torch.utils.data as DataLoader
loader = torch.utils.data.DataLoader(dataset=dataset,
                   batch_size=8,
                   shuffle=True,
                   drop_last=True)
len(loader), next(iter(loader))

(62,
 [tensor([[0.1183, 0.7285],
          [0.3232, 0.5363],
          [0.0819, 0.2085],
          [0.5961, 0.5661],
          [0.3091, 0.7646],
          [0.4269, 0.1226],
          [0.1948, 0.1928],
          [0.2074, 0.2608]]),
  tensor([0, 0, 0, 1, 0, 1, 1, 0])])

In [23]:
import torch.nn as nn
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc = nn.Sequential(
            nn.Linear(in_features=2, out_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32, out_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32, out_features=2),
            nn.Softmax(dim=1),
        )
        
    def forward(self, x):
        return self.fc(x)

model = Model()
model(torch.randn(8, 2)).shape
model(torch.randn(8, 2))

tensor([[0.5285, 0.4715],
        [0.5494, 0.4506],
        [0.4687, 0.5313],
        [0.5079, 0.4921],
        [0.5787, 0.4213],
        [0.5777, 0.4223],
        [0.5284, 0.4716],
        [0.5417, 0.4583]], grad_fn=<SoftmaxBackward0>)

In [31]:
def train():
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    
    loss_fun = nn.CrossEntropyLoss()
    
    model.train()
    
    for epoch in range(100):
        for i, (x, y) in enumerate(loader):
            out = model(x)
            
            loss = loss_fun(out, y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        if epoch % 20 == 0:
            acc = (out.argmax(dim=1) == y).sum().item() / len(y)
            print(epoch, loss.item(), acc)
        
    torch.save(model, 'model/1.model')

train()

0 0.3190913200378418 1.0
20 0.3312150835990906 1.0
40 0.315027117729187 1.0
60 0.314820796251297 1.0
80 0.3132622539997101 1.0


In [33]:
@torch.no_grad()
def test():
    model = torch.load('model/1.model')
    model.eval()
    x, y = next(iter(loader))
    out = model(x).argmax(dim=1)
    print(out, y)
    print(out == y)
test()

tensor([1, 0, 0, 1, 0, 1, 0, 0]) tensor([1, 0, 0, 1, 0, 1, 0, 0])
tensor([True, True, True, True, True, True, True, True])
